# Sage notebook

In [4]:
from sage.algebras.lie_algebras.bch import bch_iterator
import numpy as np
import re
import time 

In [5]:
channels = 2  # d
depth = 3  # L

# Symbolic preprocessing 

In [7]:
Lbl = LieAlgebra(RR, channels, step=depth)
basislength = len(Lbl.gens())
print(basislength)

5


In [8]:
R = PolynomialRing(RR, 2*basislength, 'm')
m = R.gens()
print(R)
L = LieAlgebra(PolynomialRing(RR, 2*basislength, 'm'), channels, step=depth)
print(L)
L.inject_variables()

Multivariate Polynomial Ring in m0, m1, m2, m3, m4, m5, m6, m7, m8, m9 over Real Field with 53 bits of precision
Free Nilpotent Lie algebra on 5 generators (1.00000000000000*X_1, 1.00000000000000*X_2, 1.00000000000000*X_12, 1.00000000000000*X_112, 1.00000000000000*X_122) over Multivariate Polynomial Ring in m0, m1, m2, m3, m4, m5, m6, m7, m8, m9 over Real Field with 53 bits of precision
Defining X_1, X_2, X_12, X_112, X_122


In [9]:
logSXn = L.from_vector(vector(R, m[basislength:]))   # x^(j) (so be summed over)
logSXn

m5*X_1 + m6*X_2 + m7*X_12 + m8*X_112 + m9*X_122

In [10]:
y = -L.from_vector(vector(R, m[:basislength]))  # y is inverse of mean (m symbols, to be computed)
y

(-m0)*X_1 + (-m1)*X_2 + (-m2)*X_12 + (-m3)*X_112 + (-m4)*X_122

In [11]:
S = sum([Z for Z in bch_iterator(y, logSXn)])   # symbolic BCH formula 
S

(-m0+m5)*X_1 + (-m1+m6)*X_2 + (0.500000000000000*m1*m5-0.500000000000000*m0*m6-m2+m7)*X_12 + (-0.0833333333333333*m0*m1*m5-0.0833333333333333*m1*m5^2+0.0833333333333333*m0^2*m6+0.0833333333333333*m0*m5*m6+0.500000000000000*m2*m5-0.500000000000000*m0*m7-m3+m8)*X_112 + (0.0833333333333333*m1^2*m5-0.0833333333333333*m0*m1*m6+0.0833333333333333*m1*m5*m6-0.0833333333333333*m0*m6^2-0.500000000000000*m2*m6+0.500000000000000*m1*m7-m4+m9)*X_122

In [12]:
Sdic = S.monomial_coefficients()       # apply dual basis (coefficients of lyndon basis elements)
Sdic

{(1,): -m0 + m5,
 (2,): -m1 + m6,
 (1, 2): 0.500000000000000*m1*m5 - 0.500000000000000*m0*m6 - m2 + m7,
 (1,
  1,
  2): -0.0833333333333333*m0*m1*m5 - 0.0833333333333333*m1*m5^2 + 0.0833333333333333*m0^2*m6 + 0.0833333333333333*m0*m5*m6 + 0.500000000000000*m2*m5 - 0.500000000000000*m0*m7 - m3 + m8,
 (1,
  2,
  2): 0.0833333333333333*m1^2*m5 - 0.0833333333333333*m0*m1*m6 + 0.0833333333333333*m1*m5*m6 - 0.0833333333333333*m0*m6^2 - 0.500000000000000*m2*m6 + 0.500000000000000*m1*m7 - m4 + m9}

In [13]:
Sval = list(Sdic.values())                # derive polynomials 
for i in range(len(Sval)):
    Sval[i] += m[i]
Sval

[m5,
 m6,
 0.500000000000000*m1*m5 - 0.500000000000000*m0*m6 + m7,
 -0.0833333333333333*m0*m1*m5 - 0.0833333333333333*m1*m5^2 + 0.0833333333333333*m0^2*m6 + 0.0833333333333333*m0*m5*m6 + 0.500000000000000*m2*m5 - 0.500000000000000*m0*m7 + m8,
 0.0833333333333333*m1^2*m5 - 0.0833333333333333*m0*m1*m6 + 0.0833333333333333*m1*m5*m6 - 0.0833333333333333*m0*m6^2 - 0.500000000000000*m2*m6 + 0.500000000000000*m1*m7 + m9]

In [14]:
def string_preproc(st_poly):
    result = re.sub(r'\^', '**',   st_poly)
    result = re.sub(r'()m([0-9]*)()', r'\1m[\2]\3', result)
    return "lambda m : "+result

In [15]:
outp_0 = [string_preproc(str(a)) for a in Sval]
outp_0


['lambda m : m[5]',
 'lambda m : m[6]',
 'lambda m : 0.500000000000000*m[1]*m[5] - 0.500000000000000*m[0]*m[6] + m[7]',
 'lambda m : -0.0833333333333333*m[0]*m[1]*m[5] - 0.0833333333333333*m[1]*m[5]**2 + 0.0833333333333333*m[0]**2*m[6] + 0.0833333333333333*m[0]*m[5]*m[6] + 0.500000000000000*m[2]*m[5] - 0.500000000000000*m[0]*m[7] + m[8]',
 'lambda m : 0.0833333333333333*m[1]**2*m[5] - 0.0833333333333333*m[0]*m[1]*m[6] + 0.0833333333333333*m[1]*m[5]*m[6] - 0.0833333333333333*m[0]*m[6]**2 - 0.500000000000000*m[2]*m[6] + 0.500000000000000*m[1]*m[7] + m[9]']

In [16]:
outp = [eval(a) for a in outp_0]
outp

[<function <lambda> at 0x165267be0>,
 <function <lambda> at 0x165394040>,
 <function <lambda> at 0x1653940d0>,
 <function <lambda> at 0x165394160>,
 <function <lambda> at 0x1653941f0>]

# Loading the data

In [17]:
logSX = np.load('./data.npy')
N = logSX.shape[0]

# Evaluation group mean

In [18]:
start_time = time.time()
res = np.zeros(basislength)
for k in range(basislength):
    res[k] = np.sum(outp[k](list(res)+[logSX[:,_k] for _k in range(basislength) ]))/N
print(time.time() - start_time)
print(res)

0.002259969711303711
[ 0.03478179 -0.20569871 -0.00655954 -0.00063188  0.00107996]
